In [1]:
!pip install transformers peft accelerate bitsandbytes sacremoses sentencepiece 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 42.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
model_name='mistralai/Mistral-7B-v0.1'
peft_name='Garsa3112/MentalDoctorMistral'

In [4]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

In [5]:
model=AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_8bit=True)
tokenizer=AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
peft_config=PeftConfig.from_pretrained(peft_name)
model=PeftModel.from_pretrained(model, peft_name)

adapter_config.json:   0%|          | 0.00/600 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [15]:
text="""
My boss seems to have a personal vendetta against me and does not provide me with promotion. He gave promotion to others but not me, not even a salary hike. 
"""

In [16]:
prompt=f'I am suffering from a mental problem and facing these issues: \n{text} \nPlease provide me with a solution. \nSolution:'
import gc
input_ids=tokenizer(prompt, return_tensors='pt').to(model.device)
with torch.no_grad():
    generated_ids=model.generate(**input_ids, num_beams=5, max_new_tokens=512, top_p=0.9, temperature=0.6, no_repeat_ngram_size=2)
output=tokenizer.decode(generated_ids[0], skip_special_tokens=True)
del generated_ids, input_ids
torch.cuda.empty_cache()
gc.collect()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2150

In [17]:
output=output.split('Solution:')[-1]

In [18]:
print(f'Question: {text} \nAnswer: {output}')

Question: 
My boss seems to have a personal vendetta against me and does not provide me with promotion. He gave promotion to others but not me, not even a salary hike. 
 
Answer: 
Hello, and thank you for your question. It sounds like your boss is not giving you the promotion or salary increase that you feel you deserve. This can be very frustrating, especially if you have been working hard and feel like you are not getting the recognition and compensation that is due to you. I would encourage you to talk to your supervisor about your concerns and ask if there is anything you can do to improve your performance and increase your chances of getting a promotion and/or a raise. If you don't get a positive response, it may be time to look for a new job.
